In [24]:
from selenium import webdriver
import time, urllib.request, requests, os, pickle

In [97]:
if not os.path.exists('webdev_ig'):
    os.makedirs('webdev_ig')

In [36]:
def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [40]:
likes = load_obj('likes.pkl')

In [25]:
def get_post_links(usernames):
    wd = webdriver.Chrome('chromedriver')
    posts = []
    
    for username in usernames:
        ig_link = "https://www.instagram.com/"+ username +"/"
        wd.get(ig_link)
        links = wd.find_elements_by_tag_name('a')
        for link in links:
            post = link.get_attribute('href')
            if '/p/' in post:
                posts.append(post)
    return posts

In [57]:
def get_likes_pics(posts):
    wd = webdriver.Chrome('chromedriver')
    likes = dict()
    for post in posts:
        try:
            # make request
            headers = {'User-Agent': 'Mozilla'}
            r = requests.get('{}?__a=1'.format(post), headers=headers)
            time.sleep(10)
            data = r.json()['graphql']['shortcode_media']
            shortcode = data['shortcode']
            is_video = data['is_video']
            
            if not is_video:
                # try to get likes (if possible)
                wd.get(post)
                num_likes = wd.find_element_by_partial_link_text(' likes').text
                num_likes = int(num_likes.replace(',', '').split(' ')[0])
                file_name = shortcode + '.jpg'
                likes[file_name] = num_likes

                # download image
                download_url = data['display_url']
                urllib.request.urlretrieve(download_url, 'webdev_ig/{}.jpg'.format(shortcode))
        except Exception as e:
            print(r.json())
            pass
        
    return likes

In [ ]:
# usernames whose posts are not yet included
usernames = ['gerard_sune','imagineskateboards','x3.me',
 'gochiestrella','kaleidoscope_skateboards',
 'danijenkss','skatercrossevents','skateparkmalaga',
 'chriskhan_','daggerfamilyshop'] 

In [84]:
# get post links of IG users
post_links = get_post_links(usernames)

In [85]:
# get likes of these posts
extra_likes = get_likes_pics(post_links)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [35]:
# append extra_likes information to likes
for file_name in extra_likes:
    if file_name not in likes.keys():
        likes[file_name] = extra_likes[file_name]

In [39]:
# overwrite likes.pkl with latest like info
save_obj(likes, 'likes.pkl')